# 子查询subqueries

## 子查询（subqueries）简介

子查询是指在一个 SQL 查询中嵌套另一个查询（内层查询），可以出现在 `SELECT`、`FROM`、`WHERE` 等位置。

In [ ]:
-- 示例准备：创建表并插入示例数据
DROP TABLE IF EXISTS employees;
DROP TABLE IF EXISTS departments;
CREATE TABLE departments (id INT PRIMARY KEY, name VARCHAR(50));
CREATE TABLE employees (id INT PRIMARY KEY, name VARCHAR(50), dept_id INT, salary INT);
INSERT INTO departments (id,name) VALUES (1,'Sales'),(2,'Engineering'),(3,'HR');
INSERT INTO employees (id,name,dept_id,salary) VALUES
  (1,'Alice',1,6000),
  (2,'Bob',1,4500),
  (3,'Carol',2,7000),
  (4,'Dave',2,5000),
  (5,'Eve',3,4800);

-- 子查询 SELECT id FROM departments WHERE name = 'Sales' 先找出 Sales 的 id
-- 然后外层 SELECT name FROM employees WHERE dept_id IN (...) 找出 dept_id 等于该 id 的员工并返回姓名。
SELECT name FROM employees WHERE dept_id IN (SELECT id FROM departments WHERE name = 'Sales');


# 分组group by

### 分组（GROUP BY）简介

- **作用**：将结果按一个或多个列分组，然后对每组应用聚合函数（如 COUNT、SUM、AVG、MIN、MAX、GROUP_CONCAT）。



In [ ]:

SELECT dept_id, AVG(salary) AS avg_salary
FROM employees
GROUP BY dept_id;



### ROLLUP（分级汇总）简介
在 GROUP BY 的基础上自动计算分级小计与总计，常用于报表汇总场景。使用 `WITH ROLLUP` 会在结果中附加小计/总计行。

In [ ]:
-- 输出各部门平均薪水与总平均
SELECT dept_id, AVG(salary) 
FROM employees
GROUP BY dept_id WITH ROLLUP;


### ON DELETE（删除时的外键行为）

- **CASCADE**：当父表记录被删除时，自动删除子表中相关的记录（级联删除）。
- **SET NULL**：当父表记录被删除时，将子表外键列设为 NULL（外键列必须允许 NULL）。

In [ ]:
-- 示例：CASCADE（级联删除）
DROP TABLE IF EXISTS orders_cascade;
DROP TABLE IF EXISTS customers_cascade;
CREATE TABLE customers_cascade (
	id INT PRIMARY KEY, 
	name VARCHAR(50)
	);
CREATE TABLE orders_cascade (
	id INT PRIMARY KEY AUTO_INCREMENT,
	customer_id INT,
	item VARCHAR(50),
	FOREIGN KEY (customer_id) REFERENCES customers_cascade(id) ON DELETE CASCADE
	);
INSERT INTO customers_cascade (id, name) 
VALUES 
	(1,'Alice'), 
	(2,'Bob');

INSERT INTO orders_cascade (customer_id, item) 
VALUES
 	(1,'A'), 
	(1,'B'),
	(2,'C');
-- 删除 customer 1，会自动删除其订单
DELETE FROM customers_cascade WHERE id = 1;
SELECT * FROM orders_cascade; -- 仅剩 customer_id = 2 的订单

### 存储过程（Stored Procedures）简介

- **定义**：存储过程是保存在数据库服务器上的可执行 SQL 块，通过 `CALL` 调用，常用于封装复杂查询、事务或重复逻辑。
- **特点**：支持 `IN` / `OUT` / `INOUT` 参数、条件/循环、游标（cursor）和事务控制。
- **注意**：定义过程中需要临时修改分隔符（`DELIMITER`）以避免与语句分隔符冲突；执行存储过程需要相应权限（`CREATE ROUTINE` / `EXECUTE`）；存储过程不易像普通 SQL 那样方便地进行版本控制与调试。

下面给出几个简短、可执行的示例：

In [ ]:

CREATE PROCEDURE get_avg_by_dept()
BEGIN
	SELECT dept_id, AVG(salary)
	FROM employees
	GROUP BY dept_id WITH ROLLUP;
END;

-- 调用：
CALL get_avg_by_dept();


# trigger

### 触发器（Triggers）简介

- **定义**：触发器是在表发生 `INSERT`、`UPDATE`、`DELETE` 时自动执行的数据库端代码块
- **时机**：按时机可分为 `BEFORE`（在操作前）和 `AFTER`（在操作后）。

In [ ]:
-- 创建审计表用于记录员工变更
DROP TABLE IF EXISTS test_trigger;
CREATE TABLE test_trigger (
  id INT PRIMARY KEY AUTO_INCREMENT,
  hourly_pay INT,
  work_hours INT,
  year_pay INT
);

-- 年收入计算的触发器，在插入前计算 year_pay
CREATE TRIGGER trigger_before_insert
BEFORE INSERT ON test_trigger
FOR EACH ROW
SET NEW.year_pay = NEW.hourly_pay * NEW.work_hours;

-- 测试插入
INSERT INTO test_trigger (hourly_pay, work_hours) VALUES (20, 2000);
SELECT * FROM test_trigger;